In [1]:
import urllib.request

In [2]:
url = ("https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x10cab0f50>)

In [3]:
with open("the-verdict-pl.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Łączna liczba znaków: {len(raw_text)}")
print(raw_text[:99])

Łączna liczba znaków: 20533
Zawsze uważałem Jacka Gisburna za raczej taniego geniusza – choć skądinąd poczciwego faceta – nie b


In [ ]:
#Prosty podział tekstu z regex
import re

text = "Witaj świecie. To jest test."
result = re.split(r"(\s)", text)
print(result)

['Witaj', ' ', 'świecie.', ' ', 'To', ' ', 'jest', ' ', 'test.']


In [5]:
result = re.split(r"([,.]|\s)", text)
print(result)

['Witaj', ' ', 'świecie', '.', '', ' ', 'To', ' ', 'jest', ' ', 'test', '.', '']


In [6]:
#usunięcie białych znaków
result = [item for item in result if item.strip()]
print(result)

['Witaj', 'świecie', '.', 'To', 'jest', 'test', '.']


In [7]:
#Pełny regex filtrujący białe i specjalne znaki

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

3770


In [8]:
print(preprocessed[:30])

['Zawsze', 'uważałem', 'Jacka', 'Gisburna', 'za', 'raczej', 'taniego', 'geniusza', '–', 'choć', 'skądinąd', 'poczciwego', 'faceta', '–', 'nie', 'było', 'więc', 'dla', 'mnie', 'wielkim', 'zaskoczeniem', ',', 'gdy', 'dowiedziałem', 'się', ',', 'że', 'u', 'szczytu', 'sławy']


In [9]:
all_worlds = sorted(set(preprocessed))
vocab_size = len(all_worlds)
print(vocab_size)

1590


In [13]:
all_worlds[:10]

['!', '(', ')', ',', '.', ':', ';', '?', 'A', 'Ach']

In [14]:
vocab = {token:integer for integer, token in enumerate(all_worlds)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('(', 1)
(')', 2)
(',', 3)
('.', 4)
(':', 5)
(';', 6)
('?', 7)
('A', 8)
('Ach', 9)
('Ale', 10)
('Ani', 11)
('Biedna', 12)
('Biedny', 13)
('Bywały', 14)
('Być', 15)
('Był', 16)
('Było', 17)
('Bóg', 18)
('Carlo', 19)
('Chciała', 20)
('Chciałbym', 21)
('Chicago', 22)
('Chodź', 23)
('Choć', 24)
('Ciebie', 25)
('Claude', 26)
('Co', 27)
('Croft', 28)
('Czy', 29)
('Czyż', 30)
('Człowiek', 31)
('Cóż', 32)
('Devonshire', 33)
('Dlaczego', 34)
('Do', 35)
('Dopiero', 36)
('Dubarry', 37)
('Florencji', 38)
('Gallery', 39)
('Gdy', 40)
('Gdybym', 41)
('Gideon', 42)
('Gisburn', 43)
('Gisburna', 44)
('Grafton', 45)
('Grindle', 46)
('Grindle’a', 47)
('Grindle’u”', 48)
('Hermia', 49)
('Hermii', 50)


In [19]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [20]:
tokenizer = SimpleTokenizerV1(vocab)
text = """To ostatni, jaki namalował powiedziała pani Gisburn z wybaczalną dumą."""
ids = tokenizer.encode(text)
print(ids)

[147, 814, 3, 473, 684, 933, 835, 43, 1428, 1370, 370, 4]


In [21]:
print(tokenizer.decode(ids))

To ostatni, jaki namalował powiedziała pani Gisburn z wybaczalną dumą.


In [23]:
# test na próbce, której nie ma w danych
text = "Witaj, czy lubisz kawę?"
print(tokenizer.encode(text))

KeyError: 'Witaj'

In [24]:
all_tokens = sorted(list(set(preprocessed)))
preprocessed[:20]

['Zawsze',
 'uważałem',
 'Jacka',
 'Gisburna',
 'za',
 'raczej',
 'taniego',
 'geniusza',
 '–',
 'choć',
 'skądinąd',
 'poczciwego',
 'faceta',
 '–',
 'nie',
 'było',
 'więc',
 'dla',
 'mnie',
 'wielkim']

In [25]:
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1592


In [27]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('„temat”', 1587)
('„zabójcza', 1588)
('„zrobiony”', 1589)
('<|endoftext|>', 1590)
('<|unk|>', 1591)


In [28]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [31]:
text1 = "Witaj, czy lubisz kawę?"
text2 = "Wyszedł na skąpany w słońcu taras pałacu"
text = " <|endoftext|> ".join((text1, text2))
print(text)

Witaj, czy lubisz kawę? <|endoftext|> Wyszedł na skąpany w słońcu taras pałacu


In [32]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1591, 3, 306, 576, 1591, 7, 1590, 1591, 664, 1591, 1312, 1591, 1239, 1591]


In [33]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, czy lubisz <|unk|>? <|endoftext|> <|unk|> na <|unk|> w <|unk|> taras <|unk|>


Inne symbole specjalne:
[BOS] - beginning of sequence - początek tesktu/sekwencji
[EOS] - end of sequence - koniec sekwenci/tekstu
[PAD] - padding - przedłużanie tekstów 